In [2]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pickle
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import tensorflow as tf

Class and Class ID :-
1. Asthma
2. Bronchiectasis
3. Bronchiolitis
4. Copd
5. Healthy
6. LRTI
7. Pneumonia
8. URTI

In [3]:
df=pd.read_csv('Meta_Data_of_audio_files.csv')
df=df.drop(0)
display(df)
df_feat=df
file=df_feat['File_Path']
classid=df_feat['Class_ID']
df_feat

,Unnamed: 0,File_Path,Class_ID,Class
1,1,Audio files/Copd/104_1b1_Ar_sc_Litt3200.wav,4,Copd
2,2,Audio files/Copd/104_1b1_Ll_sc_Litt3200.wav,4,Copd
3,3,Audio files/Copd/104_1b1_Lr_sc_Litt3200.wav,4,Copd
4,4,Audio files/Copd/104_1b1_Pl_sc_Litt3200.wav,4,Copd
5,5,Audio files/Copd/104_1b1_Pr_sc_Litt3200.wav,4,Copd
...,...,...,...,...
883,883,Audio files/URTI/190_1b1_Tc_sc_Meditron.wav,8,URTI
884,884,Audio files/URTI/197_1b1_Al_sc_Meditron.wav,8,URTI
885,885,Audio files/URTI/197_1b1_Tc_sc_Meditron.wav,8,URTI
886,886,Audio files/URTI/210_1b1_Al_sc_Meditron.wav,8,URTI


,Unnamed: 0,File_Path,Class_ID,Class
1,1,Audio files/Copd/104_1b1_Ar_sc_Litt3200.wav,4,Copd
2,2,Audio files/Copd/104_1b1_Ll_sc_Litt3200.wav,4,Copd
3,3,Audio files/Copd/104_1b1_Lr_sc_Litt3200.wav,4,Copd
4,4,Audio files/Copd/104_1b1_Pl_sc_Litt3200.wav,4,Copd
5,5,Audio files/Copd/104_1b1_Pr_sc_Litt3200.wav,4,Copd
...,...,...,...,...
883,883,Audio files/URTI/190_1b1_Tc_sc_Meditron.wav,8,URTI
884,884,Audio files/URTI/197_1b1_Al_sc_Meditron.wav,8,URTI
885,885,Audio files/URTI/197_1b1_Tc_sc_Meditron.wav,8,URTI
886,886,Audio files/URTI/210_1b1_Al_sc_Meditron.wav,8,URTI


In [4]:
#for padding data
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desired width
    :return: padded array
    """
    h = array.shape[0]
    w = array.shape[1]
    a = max((xx - h) // 2,0)
    aa = max(0,xx - a - h)
    b = max(0,(yy - w) // 2)
    bb = max(yy - b - w,0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [4]:
def generate_features(y_cut):
    sr=28000
    max_size=4800 #my max audio file feature width
    stft = padding(np.abs(librosa.stft(y_cut, n_fft=255, hop_length = 512)), 128, max_size)
    MFCCs = padding(librosa.feature.mfcc(y_cut, n_fft=255, hop_length=512,n_mfcc=128),128,max_size)
    spec_centroid = librosa.feature.spectral_centroid(y=y_cut, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y_cut, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y_cut, sr=sr)
    #Now the padding part
    image = np.array([padding(normalize(spec_bw),1, max_size)]).reshape(1,max_size)
    image = np.append(image,padding(normalize(spec_centroid),1, max_size), axis=0) 
    #repeat the padded spec_bw,spec_centroid and chroma stft until they are stft and MFCC-sized
    for i in range(0,9):
        image = np.append(image,padding(normalize(spec_bw),1, max_size), axis=0)
        image = np.append(image, padding(normalize(spec_centroid),1, max_size), axis=0)
        image = np.append(image, padding(normalize(chroma_stft),12, max_size), axis=0)
    image=np.dstack((image,np.abs(stft)))
    image=np.dstack((image,MFCCs))
    return image

In [5]:
def generate_features(y_cut):
    sr=28000
    max_size=1 #my max audio file feature width
    stft = padding(np.abs(librosa.stft(y_cut, n_fft=255, hop_length = 512)), 128, max_size)
    MFCCs = padding(librosa.feature.mfcc(y_cut, n_fft=255, hop_length=512,n_mfcc=128),128,max_size)
#     spec_centroid = librosa.feature.spectral_centroid(y=y_cut, sr=sr)
#     chroma_stft = librosa.feature.chroma_stft(y=y_cut, sr=sr)
#     spec_bw = librosa.feature.spectral_bandwidth(y=y_cut, sr=sr)
#     #Now the padding part
#     image = np.array([padding(normalize(spec_bw),1, max_size)]).reshape(1,max_size)
#     image = np.append(image,padding(normalize(spec_centroid),1, max_size), axis=0) 
#     #repeat the padded spec_bw,spec_centroid and chroma stft until they are stft and MFCC-sized
#     for i in range(0,9):
#         image = np.append(image,padding(normalize(spec_bw),1, max_size), axis=0)
#         image = np.append(image, padding(normalize(spec_centroid),1, max_size), axis=0)
#         image = np.append(image, padding(normalize(chroma_stft),12, max_size), axis=0)
    image=np.array(np.abs(stft))
    image=np.dstack((image,MFCCs))
    return image
filename="Audio files/URTI/190_1b1_Tc_sc_Meditron.wav"
ys, sr = librosa.load(filename,sr=28000)  
print(filename + ' '+ 'going on')
#print(str(j)+' out of '+ str(len(filelist))+'\n')
#cut the file to signal start and end  
#y_cut=y[int(round(tstart*sr)):int(round(tend*sr))]  
#generate features & output numpy array          
data = generate_features(ys) 
data

Audio files/URTI/190_1b1_Tc_sc_Meditron.wav going on


array([[[ 1.4215833e-01, -7.4742224e+02],
        [ 1.6227384e+00, -7.6357361e+02],
        [ 6.4337510e-01, -7.6109424e+02],
        ...,
        [ 2.4151199e+00, -7.6927966e+02],
        [ 1.3649483e+00, -7.7500647e+02],
        [ 4.0880399e+00, -7.6443835e+02]],

       [[ 1.4701825e-01,  4.8166626e+01],
        [ 8.8302743e-01,  3.4645660e+01],
        [ 3.3674550e-01,  3.8118698e+01],
        ...,
        [ 1.2071362e+00,  2.7018990e+01],
        [ 6.8681067e-01,  1.9909580e+01],
        [ 2.0864837e+00,  3.4205540e+01]],

       [[ 1.2648177e-01,  2.6522915e+01],
        [ 6.5607674e-02,  2.9652164e+01],
        [ 1.2338499e-01,  3.2846474e+01],
        ...,
        [ 8.4471829e-02,  2.3336056e+01],
        [ 1.9417366e-02,  1.8612392e+01],
        [ 1.5330699e-01,  3.1055622e+01]],

       ...,

       [[ 1.2660849e-04, -3.2081327e+00],
        [ 8.3566702e-06, -9.4171238e-01],
        [ 2.0882509e-05, -8.7942886e-01],
        ...,
        [ 1.9664827e-05, -1.1132498e+00],
     

In [15]:
# def get_features(df_in):   
#     features=[]     
#     labels = [] #empty array to store labels     
#     #For each species, determine how many augmentations are needed
#     df_in=df_in.reset_index()     
#     for i in df_in.Class_ID.unique():
#            print('Class_ID:',i)    
#            #all the file indices with the same species_id     
#            filelist = df_in.loc[df_in.Class_ID == i].index         
#            #filelist = df_in.loc[df_in.Class_ID == 3].index
#            for j in range(0,len(filelist)):             
#                         filename = df_in.iloc[filelist[j]].File_Path #get the filename   
#                         #define the beginning time of the signal          
#                         #tstart = df_in.iloc[filelist[j]].t_min             
#                         #tend = df_in.iloc[filelist[j]].t_max #end of signal
#                         #recording_id = df_in.iloc[filelist[j]].recording_id
#                         Class_ID = i
#                         #songtype_id = df_in.iloc[filelist[j]].songtype_id   
#                         #Load the file
#                         ys, sr = librosa.load(filename,sr=28000)  
#                         print(filename + ' '+ 'going on')
#                         print(str(j)+' out of '+ str(len(filelist))+'\n')
#                         #cut the file to signal start and end  
#                         #y_cut=y[int(round(tstart*sr)):int(round(tend*sr))]  
#                         #generate features & output numpy array          
#                         data = generate_features(ys) 
#                         features.append(data[np.newaxis,...])    
#                         labels.append(Class_ID)     
#     output=np.concatenate(features,axis=0)     
#     return(np.array(output), labels)

In [6]:
def get_features(df_in):   
    features=[]     
    labels = [] #empty array to store labels     
    #For each species, determine how many augmentations are needed
    df_in=df_in.reset_index()     
    for i in [8]:#df_in.Class_ID.unique():
           print('Class_ID:',i)    
           #all the file indices with the same species_id     
           filelist = df_in.loc[df_in.Class_ID == i].index         
           #filelist = df_in.loc[df_in.Class_ID == 3].index
           for j in range(0,len(filelist)):             
                        filename = df_in.iloc[filelist[j]].File_Path #get the filename   
                        #define the beginning time of the signal          
                        #tstart = df_in.iloc[filelist[j]].t_min             
                        #tend = df_in.iloc[filelist[j]].t_max #end of signal
                        #recording_id = df_in.iloc[filelist[j]].recording_id
                        Class_ID = i
                        #songtype_id = df_in.iloc[filelist[j]].songtype_id   
                        #Load the file
                        ys, sr = librosa.load(filename,sr=28000)  
                        print(filename + ' '+ 'going on')
                        print(str(j)+' out of '+ str(len(filelist))+'\n')
                        #cut the file to signal start and end  
                        ys=ys[int(round(0*sr)):int(round(10*sr))]  
                        #generate features & output numpy array          
                        data = generate_features(ys) 
                        features.append(data[np.newaxis,...])    
                        labels.append(Class_ID)     
    X=np.concatenate(features,axis=0)     
    y=labels
    return 0

In [15]:
def get_features(df_in):   
    features=[]     
    labels = [] #empty array to store labels     
    #For each species, determine how many augmentations are needed
    df_in=df_in.reset_index()     
    for i in [8]:#df_in.Class_ID.unique():
           print('Class_ID:',i)    
           #all the file indices with the same species_id     
           filelist = df_in.loc[df_in.Class_ID == i].index         
           #filelist = df_in.loc[df_in.Class_ID == 3].index
           for j in range(0,len(filelist)):             
                        filename = df_in.iloc[filelist[j]].File_Path #get the filename   
                        #define the beginning time of the signal          
                        #tstart = df_in.iloc[filelist[j]].t_min             
                        #tend = df_in.iloc[filelist[j]].t_max #end of signal
                        #recording_id = df_in.iloc[filelist[j]].recording_id
                        Class_ID = i
                        #songtype_id = df_in.iloc[filelist[j]].songtype_id   
                        #Load the file
                        ys, sr = librosa.load(filename,sr=28000)  
                        print(filename + ' '+ 'going on')
                        print(str(j)+' out of '+ str(len(filelist))+'\n')
                        #cut the file to signal start and end  
                        #y_cut=y[int(round(tstart*sr)):int(round(tend*sr))]  
                        #generate features & output numpy array          
                        data = generate_features(ys) 
                        features.append(data[np.newaxis,...])    
                        labels.append(Class_ID)     
    X=np.concatenate(features,axis=0)     
    y=labels
    return 0

In [16]:
X=[]
y=[]
get_features(pd.concat([file, classid], axis=1), X, y)

Class_ID: 8
Audio files/URTI/101_1b1_Al_sc_Meditron.wav going on
0 out of 23

Audio files/URTI/101_1b1_Pr_sc_Meditron.wav going on
1 out of 23

Audio files/URTI/105_1b1_Tc_sc_Meditron.wav going on
2 out of 23

Audio files/URTI/119_1b1_Ar_sc_Meditron.wav going on
3 out of 23

Audio files/URTI/129_1b1_Ar_sc_Meditron.wav going on
4 out of 23

Audio files/URTI/131_1b1_Al_sc_Meditron.wav going on
5 out of 23

Audio files/URTI/137_1b1_Ar_sc_Meditron.wav going on
6 out of 23

Audio files/URTI/137_1b1_Ll_sc_Meditron.wav going on
7 out of 23

Audio files/URTI/148_1b1_Al_sc_Meditron.wav going on
8 out of 23

Audio files/URTI/150_1b2_Al_sc_Meditron.wav going on
9 out of 23

Audio files/URTI/164_1b1_Ll_sc_Meditron.wav going on
10 out of 23

Audio files/URTI/165_1b1_Ar_sc_Meditron.wav going on
11 out of 23

Audio files/URTI/165_1b1_Pl_sc_Meditron.wav going on
12 out of 23

Audio files/URTI/165_1b1_Pr_sc_Meditron.wav going on
13 out of 23

Audio files/URTI/188_1b1_Al_sc_Meditron.wav going on
14 out 

0

In [2]:
y=np.load('y.npy')
y

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [3]:
X=np.load('X.npy')

In [7]:
X10, y10=get_features(pd.concat([file, classid], axis=1))

Class_ID: 8
Audio files/URTI/101_1b1_Al_sc_Meditron.wav going on
0 out of 23

Audio files/URTI/101_1b1_Pr_sc_Meditron.wav going on
1 out of 23

Audio files/URTI/105_1b1_Tc_sc_Meditron.wav going on
2 out of 23

Audio files/URTI/119_1b1_Ar_sc_Meditron.wav going on
3 out of 23

Audio files/URTI/129_1b1_Ar_sc_Meditron.wav going on
4 out of 23

Audio files/URTI/131_1b1_Al_sc_Meditron.wav going on
5 out of 23

Audio files/URTI/137_1b1_Ar_sc_Meditron.wav going on
6 out of 23

Audio files/URTI/137_1b1_Ll_sc_Meditron.wav going on
7 out of 23

Audio files/URTI/148_1b1_Al_sc_Meditron.wav going on
8 out of 23

Audio files/URTI/150_1b2_Al_sc_Meditron.wav going on
9 out of 23

Audio files/URTI/164_1b1_Ll_sc_Meditron.wav going on
10 out of 23

Audio files/URTI/165_1b1_Ar_sc_Meditron.wav going on
11 out of 23

Audio files/URTI/165_1b1_Pl_sc_Meditron.wav going on
12 out of 23

Audio files/URTI/165_1b1_Pr_sc_Meditron.wav going on
13 out of 23

Audio files/URTI/188_1b1_Al_sc_Meditron.wav going on
14 out 

TypeError: cannot unpack non-iterable int object